In [40]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import CRUD operations via the animalShelter class from CRUD.py
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Mongo server access information
username = 'aacuser'
password = 'Chunk1234'
host = 'nv-desktop-services.apporto.com'
port = 30061
database = 'AAC'
collection = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter()


df = pd.DataFrame(columns=[
    "animal_id", "name", "breed", "age_upon_outcome_in_weeks", "sex_upon_outcome", 
    "animal_type", "color", "outcome_type", "date_of_birth", "outcome_subtype",
    "location_lat", "location_long", "outcome_month", "outcome_year"
])


## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
# App initialization
app = JupyterDash(__name__)

# Load logo
image_filename = 'gslogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App layout
app.layout = html.Div(style={
    'fontFamily': 'Segoe UI, sans-serif',
    'padding': '20px'
}, children=[

    # Logo & title
    html.Div([
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                id='gs-logo-tjones-0425',
                style={'height': '150px', 'cursor': 'pointer'}
            ),
            href='https://www.snhu.edu',
            target='_blank'
        ),
        html.H1("Grazioso Salvare Dashboard | CS340 | Project 2", style={'marginTop': '10px'}),
        html.P("Created by Taylor Jones", style={'fontStyle': 'italic', 'fontSize': '16px'})
    ], style={'textAlign': 'center'}),

    html.Hr(),

    # Rescue type filter
    html.Div([
        html.Label("Rescue Type Filter", style={'fontWeight': 'bold'}),
        dcc.RadioItems(
            id='rescue-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster/Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block', 'margin': '6px'}
        )
    ], style={
        'padding': '20px',
        'marginBottom': '20px',
        'width': '30%',
        'marginLeft': 'auto',
        'marginRight': 'auto',        
    }),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=[],
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px',
            'width': '100px',
            'maxWidth': '180px',
            'whiteSpace': 'normal'},
        page_action="native",
        page_current=0,
        page_size=10
    ),

    html.Br(),
    html.Hr(),

    # Graph and map side-by-side
    html.Div(className='row', style={'display': 'flex', 'gap': '30px', 'justifyContent': 'center'}, children=[
        html.Div(id='graph-id', className='col s12 m6', style={'width': '45%', 'textAlign': 'center'}),
        html.Div(id='map-id', className='col s12 m6', style={'width': '45%'})
    ])
])

# Highlight columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Rescue type filtering
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-filter', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Disaster Rescue or Individual Tracking':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}  # Reset/default

    # query the DB
    data = pd.DataFrame.from_records(db.read(query))
    data.drop_duplicates(subset="animal_id", inplace=True)

    if '_id' in data.columns:
        data.drop(columns=['_id'], inplace=True)
    
    return data.to_dict('records')

# Pie chart (top breeds)
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.P("No data to display.")
    
    dff = pd.DataFrame(viewData)
    dff = dff[dff['breed'].notna() & (dff['breed'] != '')]
    top_breeds = dff['breed'].value_counts().nlargest(10).index
    dff = dff[dff['breed'].isin(top_breeds)]

    return [
        dcc.Graph(
            figure=px.pie(
                dff,
                names='breed',
                title='Top 10 Breeds in View'
            )
        )
    ]

# Map (updates from selected row)
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run the app
app.run_server(debug=True)

Dash app running on http://127.0.0.1:10672/
